In [26]:
# Importation des bibliothèques nécessaires
from keras.preprocessing.image import img_to_array  # Convertir une image en tableau NumPy
from sklearn.utils import shuffle  # Mélanger les données
from sklearn.preprocessing import LabelBinarizer  # Convertir les étiquettes en format binaire
from sklearn.model_selection import train_test_split  # Diviser les données en ensembles d'entraînement et de test
from keras.preprocessing.image import ImageDataGenerator  # Génération d'images augmentées
from sklearn.metrics import classification_report, confusion_matrix  # Évaluation des performances du modèle
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img  # Traitement d'images avec Keras


In [2]:
train_data_dir = '/kaggle/input/agriculture-crop-images/kag2'

**Data Augmentation Configuration and Training Data Generator Initialization**


In [3]:
train_datagen = ImageDataGenerator(rescale=1.0/255.0, horizontal_flip=True,vertical_flip=True, rotation_range=90)
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(224,224),
        batch_size=64,
        class_mode='categorical',
        shuffle=True)

Found 804 images belonging to 5 classes.


In [4]:
train_generator.class_indices


{'jute': 0, 'maize': 1, 'rice': 2, 'sugarcane': 3, 'wheat': 4}

**Model Training VGG16**


In [5]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model

**Initialization of VGG16 Pre-trained Model and Freezing Layers**


In [6]:
pre_model = VGG16(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (224, 224, 3))

for layer in pre_model.layers:
    layer.trainable = False              

58889256/58889256 [==============================] - 0s 0us/step


In [7]:
pre_model.summary()


Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

**Adding Flatten and Final Dense Layer to VGG16 Pre-trained Model**

In [8]:
last_layer = Flatten()(pre_model.output)
final_layer = Dense(5, activation='softmax')(last_layer)

In [9]:
model = Model(inputs=pre_model.input, outputs=final_layer)


In [10]:
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

**Compiling the Model**

In [11]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics='accuracy')


In [12]:
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [13]:
model.fit_generator(train_generator, epochs=10)


/tmp/ipykernel_41/2975305634.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, epochs=10)


Epoch 1/10
13/13 [==============================] - 230s 17s/step - loss: 1.8794 - accuracy: 0.3396
Epoch 2/10
13/13 [==============================] - 228s 17s/step - loss: 1.1595 - accuracy: 0.5274
Epoch 3/10
13/13 [==============================] - 226s 17s/step - loss: 0.8594 - accuracy: 0.6741
Epoch 4/10
13/13 [==============================] - 225s 17s/step - loss: 0.6860 - accuracy: 0.7624
Epoch 5/10
13/13 [==============================] - 225s 17s/step - loss: 0.6046 - accuracy: 0.7998
Epoch 6/10
13/13 [==============================] - 225s 17s/step - loss: 0.5328 - accuracy: 0.8271
Epoch 7/10
13/13 [==============================] - 225s 17s/step - loss: 0.4641 - accuracy: 0.8644
Epoch 8/10
13/13 [==============================] - 225s 17s/step - loss: 0.4214 - accuracy: 0.8781
Epoch 9/10
13/13 [==============================] - 225s 17s/step - loss: 0.4055 - accuracy: 0.8856
Epoch 10/10
13/13 [==============================] - 225s 17s/step - loss: 0.3720 - accuracy: 0.8881

**Save Model**


In [15]:
model.save('my_model.keras')

In [16]:
import matplotlib.pyplot as plt


**Evaluating the model**

In [27]:
model.evaluate_generator(test_generator)

/tmp/ipykernel_41/2205837447.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  model.evaluate_generator(test_generator)


[0.4751569330692291, 0.8457711338996887]

In [17]:
#TestData
test_data_dir = '/kaggle/input/agriculture-crop-images/crop_images'
test_datagen = ImageDataGenerator(rescale=1./255)
 
test_batchsize = 64
 
test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(224,225),
        batch_size=64,
        class_mode='categorical',
        shuffle=False)

Found 201 images belonging to 5 classes.


In [18]:
import numpy as np
model.load_weights("my_model.keras")
class_labels = test_generator.class_indices
class_labels = {v: k for k, v in class_labels.items()}
classes = list(class_labels.values())
Y_pred = model.predict_generator(test_generator)
y_pred = np.argmax(Y_pred, axis=1)

/tmp/ipykernel_41/119025590.py:6: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  Y_pred = model.predict_generator(test_generator)


In [25]:
model.evaluate_generator(test_generator)

/tmp/ipykernel_41/2205837447.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  model.evaluate_generator(test_generator)


[0.4751569330692291, 0.8457711338996887]

In [19]:
import cv2
import numpy as np

**Some predictions**


In [20]:
def predict_crop(path,actual,class_labels):
    predict_datagen = ImageDataGenerator(rescale=1./255)
    img = cv2.imread(path)
    img = cv2.resize(img, (224, 224))
    img = np.array(img).reshape((1, 224, 224, 3))
    Y_pred = model.predict(img)
    y_pred = np.argmax(Y_pred, axis=1)
    if y_pred == actual:
        print('Correct prediction')
    else:
        print("Wrong Prediction!!")
    print('Actual class "{0}" and predicted class "{1}"'.format(class_labels[int(y_pred)],class_labels[actual]))

In [21]:
predict_crop('/kaggle/input/agriculture-crop-images/test_crop_image/jute-field.jpg',0,class_labels)

1/1 [==============================] - 0s 454ms/step
Correct prediction
Actual class "jute" and predicted class "jute"


In [22]:
predict_crop('/kaggle/input/agriculture-crop-images/test_crop_image/maize_fieldmexico.jpeg',1,class_labels)

1/1 [==============================] - 0s 347ms/step
Correct prediction
Actual class "maize" and predicted class "maize"


In [23]:
predict_crop('/kaggle/input/agriculture-crop-images/test_crop_image/rice-field.jpg',0,class_labels)

1/1 [==============================] - 0s 298ms/step
Correct prediction
Actual class "jute" and predicted class "jute"


In [24]:
predict_crop('/kaggle/input/agriculture-crop-images/test_crop_image/sugarcane fields.jpg',1,class_labels)

1/1 [==============================] - 0s 295ms/step
Correct prediction
Actual class "maize" and predicted class "maize"
